In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Comp4211_project/Project
%ls

/content/drive/MyDrive/Comp4211_project/Project
classifier/       config.py  CycleGAN_save/  dataset/         __pycache__/  test.ipynb
Classifier_save/  cyclegan/  data/           inference.ipynb  samples/      train.py


In [3]:
import torch
from classifier.classifier import Classifier
from data.dataloader import DataLoader_Classifier
from config import opt
import os
opt = opt()


In [7]:
netC = Classifier(input_size=64)
save_path = opt.classifier_save_path
check_point = 50
name_type = opt.name_T
netC.load_state_dict(torch.load(os.path.join(save_path, f"{name_type}/netC_{check_point}.pth")))
netC.eval()
netC.cuda()

dataloader = DataLoader_Classifier(img_size=64)
labels_name = dataloader.get_labels_names()
idx_to_class = {v: k for k, v in labels_name.items()}
print(idx_to_class)

{0: 'lung_aca', 1: 'lung_n', 2: 'lung_scc'}


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
from cyclegan.cyclegan import cycleD, cycleG

netG_aca_n = cycleG()
netG_scc_n = cycleG()
netG_aca_scc = cycleG()
# Load the model
check_point_aca_n = 25
check_point_scc_n = 25
check_point_aca_scc = 25
save_path = opt.gan_save_path
name_type = opt.name_T

name_A = 'lung_aca'
name_B = 'lung_n'
name_C = 'lung_scc'

netG_aca_n.load_state_dict(torch.load(os.path.join(save_path, f"generator/{name_type}/{name_A}_and_{name_B}/netG_{check_point_aca_n}.pth")))
netG_scc_n.load_state_dict(torch.load(os.path.join(save_path, f"generator/{name_type}/{name_C}_and_{name_B}/netG_{check_point_scc_n}.pth")))
netG_aca_scc.load_state_dict(torch.load(os.path.join(save_path, f"generator/{name_type}/{name_A}_and_{name_C}/netG_{check_point_aca_scc}.pth")))

netG_aca_n.eval().cuda()
netG_scc_n.eval().cuda()
netG_aca_scc.eval().cuda()

Model loaded from epoch 25


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

def transform (size):
  return transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
])

def inference(image_sc): # image path
  raw_image = Image.open(image_path).convert('RGB')
  image = transform(64).transform(raw_image).unsqueeze(0).cuda()
  output = netC(image)
  _ , predicted = torch.max(output.data,1)
  predicted_class = idx_to_class[predicted.item()]

  image = transform(96).transform(raw_image).unsqueeze(0).cuda()
  blank =  torch.zeros(1,3,96,96).cuda()
  match predicted_class:
    case 'lung_aca':
      image_aca = image
      _, image_n = netG_aca_n(image, blank)
      _, image_scc = netG_aca_scc(image,blank)
    case 'lung_scc':
      image_scc = image
      _, image_n = netG_scc_n(image, blank)
      image_aca, _ = netG_aca_scc(blank,image)
    case 'lung_n':
      image_n = image
      image_aca, _ = netG_aca_n(blank, image)
      image_scc, _ = netG_scc_n(blank, image)

  image_aca = image.cpu().detach().numpy()
  image_n = image_n.cpu().detach().numpy()
  image_scc = image_scc.cpu().detach().numpy()

  np.transpose(image_aca, (0, 2, 3, 1))
  np.transpose(image_n, (0, 2, 3, 1))
  np.transpose(image_scc, (0, 2, 3, 1))

  plt.figure(figsize=(10, 10))

  plt.subplot(3, 1, 1)
  plt.imshow(image_n[0])
  if predicted_class == 'lung_n':
    plt.title('n(real)')
  else:
    plt.title('n(fake)')

  plt.subplot(3, 1, 2)
  plt.imshow(image_aca[0])
  if predicted_class == 'lung_aca':
    plt.title('aca(real)')
  else:
    plt.title('aca(fake)')

  plt.subplot(3, 1, 3)
  plt.imshow(image_scc[0])
  if predicted_class == 'lung_scc':
    plt.title('scc(real)')
  else:
    plt.title('scc(fake)')

  return





In [ ]:
# transform = transforms.Compose([
#     transforms.Resize((96, 96)),
#     transforms.ToTensor(),
# ])

# # Load the image
# image_path = "/content/drive/MyDrive/Comp4211_tempt/Project/lungaca10.jpeg（副本）"
# image = Image.open(image_path).convert('RGB')


# # Apply the transformation
# image = transform(image).unsqueeze(0).cuda()
# blank =  torch.zeros(1,3,96,96).cuda()
# _, guess = netG(image, blank)
# guess = guess.cpu().detach().numpy()
# image = image.cpu().detach().numpy()


# guess = np.transpose(guess, (0, 2, 3, 1))
# image = np.transpose(image, (0, 2, 3, 1))



# plt.figure(figsize=(10, 10))

# plt.subplot(2, 1, 1)
# plt.imshow(image[0])
# plt.title('Real A')

# plt.subplot(2, 1, 2)
# plt.imshow(guess[0])
# plt.title('Fake B')

